In [1]:
import os

## import pyspark
import pyspark
from pyspark.sql import SparkSession

In [2]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.3.1,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog')
        .set('spark.sql.catalog.spark_catalog.type','hive')
        .set('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.iceberg.type', 'hadoop')
        .set('spark.sql.catalog.iceberg.warehouse', 'iceberg-warehouse')
)

In [3]:
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anshumanr/.ivy2/cache
The jars for the packages stored in: /Users/anshumanr/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-373781d3-5088-4c33-bf5c-8be35d24c83c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.1 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.17.178 in central
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found software.amazon.awssdk#http-client-spi;2.17.178 in central


Spark Running


# Copy On Write

In [4]:
spark.sql(''' CREATE TABLE iceberg.db.students (
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
 TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
PARTITIONED BY (class_year) ;''')

DataFrame[]

In [5]:
spark.sql('''
INSERT INTO iceberg.db.students (id, first_name, last_name, major, class_year)
VALUES
(1, 'John', 'Doe', 'Computer Science', 2023),
(2, 'Jane', 'Doe', 'Business', 2019),
(3, 'Peter', 'Smith', 'Engineering', 2021),
(4, 'Susan', 'Williams', 'Nursing', 2023),
(5, 'David', 'Johnson', 'Law', 2020),
(6, 'Elizabeth', 'Brown', 'Art', 2021),
(7, 'Michael', 'Green', 'History', 2019),
(8, 'Sarah', 'White', 'English', 2020),
(9, 'William', 'Black', 'Mathematics', 2021),
(10, 'Mary', 'Brown', 'Physics', 2022),
(11, 'Thomas', 'Green', 'Chemistry', 2022),
(12, 'Jennifer', 'White', 'Biology', 2020),
(13, 'Charles', 'Black', 'Geology', 2020),
(14, 'Lisa', 'Brown', 'Astronomy', 2019),
(15, 'Henry', 'Green', 'Meteorology', 2020),
(16, 'Nancy', 'White', 'Economics', 2022),
(17, 'Daniel', 'Black', 'Political Science', 2022),
(18, 'Emily', 'Brown', 'Philosophy', 2021),
(19, 'Matthew', 'Green', 'Psychology', 2019),
(20, 'Jessica', 'White', 'Sociology', 2021); ''')

DataFrame[]

In [6]:
#Reading 
spark.sql("select * from iceberg.db.students").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  2|      Jane|      Doe|         Business|      2019|
|  7|   Michael|    Green|          History|      2019|
| 14|      Lisa|    Brown|        Astronomy|      2019|
| 19|   Matthew|    Green|       Psychology|      2019|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 20|   Jessica|    White|        Sociology|      2021|
| 10|      Mary|    Brown|          Physics|    

In [7]:
#Updating
spark.sql("update iceberg.db.students set major = 'Zoology' where id = 4;")

DataFrame[]

In [8]:
spark.sql("select * from iceberg.db.students").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  2|      Jane|      Doe|         Business|      2019|
|  7|   Michael|    Green|          History|      2019|
| 14|      Lisa|    Brown|        Astronomy|      2019|
| 19|   Matthew|    Green|       Psychology|      2019|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 20|   Jessica|    White|        Sociology|      2021|
| 10|      Mary|    Brown|          Physics|    

In [9]:
#deleting records
spark.sql("delete from iceberg.db.students where class_year < 2020;")

DataFrame[]

In [10]:
spark.sql("select * from iceberg.db.students").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  1|      John|      Doe| Computer Science|      2023|
|  4|     Susan| Williams|          Zoology|      2023|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 20|   Jessica|    White|        Sociology|      2021|
| 10|      Mary|    Brown|          Physics|      2022|
| 11|    Thomas|    Green|        Chemistry|      2022|
| 16|     Nancy|    White|        Economics|    

In [11]:
#Creating a new table used to merge records
spark.sql(''' CREATE TABLE iceberg.db.new_table(
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
''')

DataFrame[]

In [12]:
spark.sql('''
INSERT INTO iceberg.db.new_table(id, first_name, last_name, major, class_year)
VALUES
(1, 'James', 'smith', 'Commerce', 2023),
(2, 'Jane', 'Foster', 'Astrology', 2018),
(20, 'Peter', 'Parker', 'Aerospace', 2019); ''')

DataFrame[]

In [13]:
spark.sql("select * from iceberg.db.new_table").show()

+---+----------+---------+---------+----------+
| id|first_name|last_name|    major|class_year|
+---+----------+---------+---------+----------+
|  1|     James|    smith| Commerce|      2023|
|  2|      Jane|   Foster|Astrology|      2018|
| 20|     Peter|   Parker|Aerospace|      2019|
+---+----------+---------+---------+----------+



In [14]:
#merging records from new_table into students

In [15]:
spark.sql('''Merge into iceberg.db.students
using (select * from iceberg.db.new_table) nt 
on iceberg.db.students.id = nt.id
when matched then update set 
iceberg.db.students.id = nt.id,
iceberg.db.students.first_name = nt.first_name,
iceberg.db.students.last_name = nt.last_name,
iceberg.db.students.major = nt.major,
iceberg.db.students.class_year = nt.class_year
when not matched then insert *;
''')

DataFrame[]

In [16]:
spark.sql("select * from iceberg.db.students").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  2|      Jane|   Foster|        Astrology|      2018|
| 20|     Peter|   Parker|        Aerospace|      2019|
| 10|      Mary|    Brown|          Physics|      2022|
| 11|    Thomas|    Green|        Chemistry|      2022|
| 16|     Nancy|    White|        Economics|      2022|
| 17|    Daniel|    Black|Political Science|      2022|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|    

In [17]:
#inspecting table history
df=spark.sql("SELECT * FROM iceberg.db.students.history")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students/history",header=True)

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-08-17 15:07:...|5837853119636701363|               null|               true|
|2023-08-17 15:07:...| 934080563060979373|5837853119636701363|               true|
|2023-08-17 15:07:...|7587528544806105650| 934080563060979373|               true|
|2023-08-17 15:08:...|6014490099645444523|7587528544806105650|               true|
+--------------------+-------------------+-------------------+-------------------+



In [18]:
#inspecting table snapshots
df=spark.sql("SELECT * FROM iceberg.db.students.snapshots")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students/snapshots",header=True)

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2023-08-17 15:07:...|5837853119636701363|               null|   append|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:07:...| 934080563060979373|5837853119636701363|overwrite|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:07:...|7587528544806105650| 934080563060979373|   delete|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:08:...|6014490099645444523|7587528544806105650|overwrite|iceberg-warehouse...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



# Merge On Read

In [19]:
spark.sql(''' CREATE TABLE iceberg.db.students2 (
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
 TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read',
    'format-version'='2'
)
PARTITIONED BY (class_year) ;''')

DataFrame[]

In [20]:
spark.sql('''
INSERT INTO iceberg.db.students2 (id, first_name, last_name, major, class_year)
VALUES
(1, 'John', 'Doe', 'Computer Science', 2023),
(2, 'Jane', 'Doe', 'Business', 2019),
(3, 'Peter', 'Smith', 'Engineering', 2021),
(4, 'Susan', 'Williams', 'Nursing', 2023),
(5, 'David', 'Johnson', 'Law', 2020),
(6, 'Elizabeth', 'Brown', 'Art', 2021),
(7, 'Michael', 'Green', 'History', 2019),
(8, 'Sarah', 'White', 'English', 2020),
(9, 'William', 'Black', 'Mathematics', 2021),
(10, 'Mary', 'Brown', 'Physics', 2022),
(11, 'Thomas', 'Green', 'Chemistry', 2022),
(12, 'Jennifer', 'White', 'Biology', 2020),
(13, 'Charles', 'Black', 'Geology', 2020),
(14, 'Lisa', 'Brown', 'Astronomy', 2019),
(15, 'Henry', 'Green', 'Meteorology', 2020),
(16, 'Nancy', 'White', 'Economics', 2022),
(17, 'Daniel', 'Black', 'Political Science', 2022),
(18, 'Emily', 'Brown', 'Philosophy', 2021),
(19, 'Matthew', 'Green', 'Psychology', 2019),
(20, 'Jessica', 'White', 'Sociology', 2021); ''')

DataFrame[]

In [21]:
#Reading 
spark.sql("select * from iceberg.db.students2").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  2|      Jane|      Doe|         Business|      2019|
|  7|   Michael|    Green|          History|      2019|
| 14|      Lisa|    Brown|        Astronomy|      2019|
| 19|   Matthew|    Green|       Psychology|      2019|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 20|   Jessica|    White|        Sociology|      2021|
| 10|      Mary|    Brown|          Physics|    

In [22]:
#Updating
spark.sql("update iceberg.db.students2 set last_name = 'Jones' where id = 20;")

DataFrame[]

In [23]:
spark.sql("select * from iceberg.db.students2").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  2|      Jane|      Doe|         Business|      2019|
|  7|   Michael|    Green|          History|      2019|
| 14|      Lisa|    Brown|        Astronomy|      2019|
| 19|   Matthew|    Green|       Psychology|      2019|
| 20|   Jessica|    Jones|        Sociology|      2021|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 10|      Mary|    Brown|          Physics|    

In [24]:
#deleting records
spark.sql("delete from iceberg.db.students2 where class_year < 2020;")

DataFrame[]

In [25]:
spark.sql("select * from iceberg.db.students2").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
| 20|   Jessica|    Jones|        Sociology|      2021|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 10|      Mary|    Brown|          Physics|      2022|
| 11|    Thomas|    Green|        Chemistry|      2022|
| 16|     Nancy|    White|        Economics|      2022|
| 17|    Daniel|    Black|Political Science|      2022|
|  1|      John|      Doe| Computer Science|    

In [26]:
spark.sql('select * from iceberg.db.new_table').show()

+---+----------+---------+---------+----------+
| id|first_name|last_name|    major|class_year|
+---+----------+---------+---------+----------+
|  1|     James|    smith| Commerce|      2023|
|  2|      Jane|   Foster|Astrology|      2018|
| 20|     Peter|   Parker|Aerospace|      2019|
+---+----------+---------+---------+----------+



In [27]:
#merging new_table into students2
spark.sql('''Merge into iceberg.db.students2
using (select * from iceberg.db.new_table) nt 
on iceberg.db.students2.id = nt.id
when matched then update set 
iceberg.db.students2.id = nt.id,
iceberg.db.students2.first_name = nt.first_name,
iceberg.db.students2.last_name = nt.last_name,
iceberg.db.students2.major = nt.major,
iceberg.db.students2.class_year = nt.class_year
when not matched then insert *;
''')

DataFrame[]

In [28]:
spark.sql("select * from iceberg.db.students2").show()

+---+----------+---------+-----------------+----------+
| id|first_name|last_name|            major|class_year|
+---+----------+---------+-----------------+----------+
|  2|      Jane|   Foster|        Astrology|      2018|
| 20|     Peter|   Parker|        Aerospace|      2019|
|  5|     David|  Johnson|              Law|      2020|
|  8|     Sarah|    White|          English|      2020|
| 12|  Jennifer|    White|          Biology|      2020|
| 13|   Charles|    Black|          Geology|      2020|
| 15|     Henry|    Green|      Meteorology|      2020|
|  1|     James|    smith|         Commerce|      2023|
|  3|     Peter|    Smith|      Engineering|      2021|
|  6| Elizabeth|    Brown|              Art|      2021|
|  9|   William|    Black|      Mathematics|      2021|
| 18|     Emily|    Brown|       Philosophy|      2021|
| 10|      Mary|    Brown|          Physics|      2022|
| 11|    Thomas|    Green|        Chemistry|      2022|
| 16|     Nancy|    White|        Economics|    

In [29]:
#inspecting table history
df=spark.sql("SELECT * FROM iceberg.db.students2.history")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students2/history",header=True)

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-08-17 15:08:...|1902236467335246869|               null|               true|
|2023-08-17 15:08:...|1033710749508415506|1902236467335246869|               true|
|2023-08-17 15:08:...|7716988095302758307|1033710749508415506|               true|
|2023-08-17 15:08:...|5293719449446274574|7716988095302758307|               true|
+--------------------+-------------------+-------------------+-------------------+



In [30]:
#inspecting table snapshots
df=spark.sql("SELECT * FROM iceberg.db.students2.snapshots")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students2/snapshots",header=True)

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2023-08-17 15:08:...|1902236467335246869|               null|   append|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:08:...|1033710749508415506|1902236467335246869|overwrite|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:08:...|7716988095302758307|1033710749508415506|   delete|iceberg-warehouse...|{spark.app.id -> ...|
|2023-08-17 15:08:...|5293719449446274574|7716988095302758307|overwrite|iceberg-warehouse...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+

